## Table of Contents
- [Preprocessing 1D **Microphone Data** for Conversion to 2D Image Format](#Preprocessing-1D-%2A%2AMicrophone-Data%2A%2A-for-Conversion-to-2D-Image-Format)
  - [Background:](#Background%3A)
  - [Read raw microphone data](#Read-raw-microphone-data)
  - [Removing noise through STFT and FFT filtering](#Removing-noise-through-STFT-and-FFT-filtering)
- [🏠 Home](../../../../../welcomePage.ipynb)

# Preprocessing 1D **Microphone Data** for Conversion to 2D Image Format

## Background:

**A method to filter raw microphone signal is demonstrated in this section.**
In this work, the airborne acoustic emission signal of the process zone is captured by a free-field standard microphone (G.R.A.S.46AE 1/2′′ CCP), with a frequency range of 3.15 Hz–20 kHz (±2 dB) and a sensitivity of 50 mV/Pa. The microphone is installed inside the build chamber at an angle of about 45◦ facing the center of the building zone. The distance from the laser scanning area to the
microphone is approximately 200 mm. The microphone is connected via a NI-9218 data acquisition card. The overall monitoring system is controlled by the LabView software, which provides convenience for adjusting the acquisition parameters.

<img src="./figures/2.png" alt="Drawing" style="width: 400px;" title=" Experimental setup and a schematic representation of the SLM system"/>

*Experimental setup and a schematic representation of the SLM system (The microphone sensor in red circle)*

### Process detail

1. **Short-time Fourier transform (STFT)**. The raw acoustic signal was first analyzed by the STFT to extract the time frequency information, aiming to remove the environment noise. The STFT operation was carried out in the Origin software using the signals processing tool, and the default hanning window function with a default length of 256 was applied to deal with the acoustic signals.
2. **High pass FFT filtering**. A 4 kHz high pass FFT filtering was employed to remove the low frequency environmental noise.
3. **Noise removal**. The acoustic signal of the processing and non-processing can be easily distinguished after the FFT filtering. Consequently, the interference of background noise was removed and the acoustic emission signal during the laser melting process was extracted for the subsequent analysis.
5. The raw 1D acoustic or photodiode signal with a length of $M^2$ is extracted through a running window, where $M^2$ is the size of the
converted image. The running window is selected by considering that it can not only increase the number of training samples but also help train the quality monitoring models with local sensor information. Then, the running window with $M^2$ samples in the time domain fulfills the pixels of the image by sequence. Detailly, consecutive 1D signal samples with a length of $M$ fulfill rows of the converted image in sequence. For the $L(i), i = 1, 2, …, M^2$ sample in the 1D signal, its coordinate corresponding to the pixel in the image pixel is $P(j,k)$.

$P(j, k)=\operatorname{round}\left\{\frac{L((j-1) \times M+k)-\min (L)}{\max (L)-\min (L)} \times 255\right\},(j, k=1,2, \ldots, M)$

### The information users need to provide:
1. The raw photodiode data local directory path.
2. The converted image data save folder local directory path.
3. The number of sample.
4. The length of each image.
5. The sliding step size

## Read raw microphone data

Provide the raw microphone data local directory path in the following text box, the raw data will be loaded.

(*Example*: ../data/microphone)

#### Press ▶️ to Select Raw Microphone Data to Load

In [ ]:
import os
from scipy.io import loadmat
from ipywidgets import interact
import ipywidgets as widgets
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Create text input widget for path input
path_input = widgets.Text(
    description='Directory Path:',
    placeholder='Paste or type the directory path here',
    style={'description_width': 'initial'}
)

# Create a button to confirm the directory selection
confirm_button = widgets.Button(description="Confirm Path")

text = widgets.Text(value="../data/microphone", description="Example")

# Variable to hold the path
# Default path: ../Data/Microphone
global directory_path
directory_path = ""

data_dict = {}

# Define button click event handler
def on_confirm_button_clicked(b):
    global directory_path
    directory_path = path_input.value
    print(f"Path selected: {directory_path}")
    for filename in os.listdir(directory_path):
        if filename.endswith('.mat'):
            file_path = os.path.join(directory_path, filename)
            mat_data = loadmat(file_path)
            
            data_name = os.path.splitext(filename)[0]
            print(data_name+' is loaded')
            data_dict[data_name] = mat_data
            
confirm_button.on_click(on_confirm_button_clicked)

# Display widgets
display(text)
display(path_input, confirm_button)

#### Press ▶️ to Display the Selected Raw Data Sample

In [ ]:
x = list(data_dict.keys())[0]
y = list(data_dict[x].keys())[3]

plt.figure(figsize=(8,5),dpi=100)
plt.plot(data_dict[x][y])
plt.title("Raw microphone signal sample")
plt.show()

## Removing noise through STFT and FFT filtering

Provide the image data **save folder directory path**. **The number of sample**, **length of each image**, **sliding step size**, and **the number of the image** can be modified to preview the output of the image data. Once all the variables are determined, the user can convert the signal data to image data.

(*Example*: ../data/image_data/microphone/)

### The 2-D microphone image data modification and visualization

#### Press ▶️ to Select the Save Folder

In [ ]:
# Create text input widget for path input
path_input = widgets.Text(
    description='Save Forlder Directory Path:',
    placeholder='Paste or type the directory path here',
    style={'description_width': 'initial'}
)

# Create a button to confirm the directory selection
confirm_button = widgets.Button(description="Confirm Path")
text = widgets.Text(value="../data/image_data/microphone/", description="Example")

# Variable to hold the path
# Default path: 
# /Users/zyichao/Desktop/phd/Codes_Additive_Manufacturing/Two_Sensor_Fusion_JMP/Data/photodiode
global save_folder
save_folder = ""

# Define button click event handler
def on_confirm_button_clicked(b):
    global save_folder
    save_folder = path_input.value
    print(f"Path selected: {directory_path}")
            
confirm_button.on_click(on_confirm_button_clicked)

# Display widgets
display(text)
display(path_input, confirm_button)

#### Press ▶️ to Modify the Preprocessing Properties

In [ ]:
def microphone(a,b,c,d):
    ## a:file position, 
    ##b:size of each image, c:Sliding step size, d:the number of the image
    
    filename = list(data_dict.keys())[a]
    parts = filename.split('_')
    mcp_grade = int(parts[1]) 
    mcp_num = int(parts[2])
    
    filtered_data = []
    data_all = []

    for j in range(1,18):
        x = data_dict[filename]['cDAQ1Mod2ai'+str(j)]
#         print(x.shape)

        data = x

        data_1 = data.reshape(len(data),)
        N = len(data)
        fs = 10000  # Sampling frequency
        t = np.arange(1, N+1)  # Sampling moments

        # FFT of the data
        data_fft = np.fft.fft(data_1)
        df = fs / N  # Frequency sampling interval
        data_f = np.arange(0, N) * df

        # Find the indices where frequency is less than 4000 Hz
        id0 = np.where(data_f < 4000)[0]
        id0_len = len(id0)

        # Filtering to make the low frequency part as 0
        data_fft[id0] = 0
        data_fft[-id0_len+1:] = 0

        # Inverse FFT after filtering
        data_ifft = np.real(np.fft.ifft(data_fft))
        
        data_ifft = data_ifft[1000:-1000]
        
        indices = np.where(data_ifft > 0.1)[0]
        first_index = indices[0]
        last_index = indices[-1]
        
        first_third_index = int((last_index - first_index)/3) + first_index
        second_third_index = int(2*(last_index - first_index)/3) + first_index

        first_valid_indices = indices[indices < first_third_index]
        first_valid_indice = first_valid_indices[-1]
        
        second_valid_indices_start = indices[indices > first_third_index]
        second_valid_indice_start = second_valid_indices_start[0]
        second_valid_indices_end = indices[indices < second_third_index]
        second_valid_indice_end = second_valid_indices_end[-1]
        
        third_valid_indices = indices[indices > second_third_index]
        third_valid_indice = third_valid_indices[0]
        
        y = np.concatenate([data_ifft[first_index:first_valid_indice],
                            data_ifft[second_valid_indice_start:second_valid_indice_end],
                            data_ifft[third_valid_indice:last_index]])
        filtered_data.append(y)
        
    n = 32  # Size of each image

    for m in range(1, 18):  # based on each layer, 17 layers in total
        data = filtered_data[m-1]
        N = len(data)
        W = b * b  # Window size
        # c: Sliding step size, middle value 10 * 10000 // 520 = 192, range from 50 to 300
        hdcs = (N - W) // c + 1  # Number of sliding steps 

        for k in range(1, hdcs + 1):
            one_data = data[(k-1) * c: (k-1) * c + W]
            # Normalize and scale the data to 0-255
            one_data = np.round((one_data - np.min(one_data)) / (np.max(one_data) - np.min(one_data)) * 255).astype(np.uint8)
            # Reshape the image
            one_data = one_data.reshape((b, b)) 
            
            data_all.append(one_data)

    
    # e_value = 5 if e else 0
    # result = a + b + c + d + e_value
    result = filename
    print(f"The selected sample name: {result}")
    return plt.imshow(data_all[d],cmap='gray')
    
interact(microphone,
         a=widgets.IntSlider(min=0, max=len(data_dict.keys())-1, step=1, value=0, description='The number of sample:',style={'description_width': 'initial'}),
         b=widgets.IntSlider(min=32, max=128, step=1, value=0, description='Length of each image:',style={'description_width': 'initial'}),
         c=widgets.IntSlider(min=50, max=300, step=1, value=0, description='Sliding step size:',style={'description_width': 'initial'}),
         d=widgets.IntSlider(min=0, max=100, step=1, value=0, description='The number of the image:',style={'description_width': 'initial'}),

         # d=widgets.Dropdown(options=[('One', 1), ('Two', 2), ('Three', 3)], value=1, description='d:'),
         # e=widgets.Checkbox(value=False, description='Include e (value 5)')
)


### Save the modified image data

#### Press ▶️ to Save the Modified Image Data

In [ ]:
for i in range(3):
    folder_path = os.path.join(save_folder, str(i))
    try:
        os.makedirs(folder_path)
        print("Directory ", folder_path, " Created ")
    except FileExistsError:
        print("Directory ", folder_path, " already exists")

for i in range(len(data_dict.keys())):
    filename = list(data_dict.keys())[i]
    parts = filename.split('_')
    mcp_grade = int(parts[1]) 
    mcp_num = int(parts[2])
    
    filtered_data = []
    data_all = []
    for j in range(1,18):
        x = data_dict[filename]['cDAQ1Mod2ai'+str(j)]
#         print(x.shape)

        data = x

        data_1 = data.reshape(len(data),)
        N = len(data)
        fs = 10000  # Sampling frequency
        t = np.arange(1, N+1)  # Sampling moments

        # FFT of the data
        data_fft = np.fft.fft(data_1)
        df = fs / N  # Frequency sampling interval
        data_f = np.arange(0, N) * df

        # Find the indices where frequency is less than 4000 Hz
        id0 = np.where(data_f < 4000)[0]
        id0_len = len(id0)

        # Filtering to make the low frequency part as 0
        data_fft[id0] = 0
        data_fft[-id0_len+1:] = 0

        # Inverse FFT after filtering
        data_ifft = np.real(np.fft.ifft(data_fft))
        
        data_ifft = data_ifft[1000:-1000]
        
        indices = np.where(data_ifft > 0.1)[0]
        first_index = indices[0]
        last_index = indices[-1]
        
        first_third_index = int((last_index - first_index)/3) + first_index
        second_third_index = int(2*(last_index - first_index)/3) + first_index

        first_valid_indices = indices[indices < first_third_index]
        first_valid_indice = first_valid_indices[-1]
        
        second_valid_indices_start = indices[indices > first_third_index]
        second_valid_indice_start = second_valid_indices_start[0]
        second_valid_indices_end = indices[indices < second_third_index]
        second_valid_indice_end = second_valid_indices_end[-1]
        
        third_valid_indices = indices[indices > second_third_index]
        third_valid_indice = third_valid_indices[0]

        # Plot the result
        plt.plot(data_ifft)
        plt.title("The acoustic signal after FFT filtering for "+filename)
        plt.axhline(0.1,color = 'r',linestyle='--')
        plt.axhline(-0.1,color = 'r',linestyle='--')
        plt.axvline(first_index,color = 'r')
        plt.axvline(first_valid_indice,color = 'r')
        plt.axvline(second_valid_indice_start,color = 'g')
        plt.axvline(second_valid_indice_end,color = 'g')
        plt.axvline(third_valid_indice,color = 'y')
        plt.axvline(last_index,color = 'y')
        
        plt.show()
        
        y = np.concatenate([data_ifft[first_index:first_valid_indice],
                            data_ifft[second_valid_indice_start:second_valid_indice_end],
                            data_ifft[third_valid_indice:last_index]])
#         print(y.shape)
        filtered_data.append(y)
       
    folder = save_folder + str(mcp_grade)
    n = 32  # Size of each image

    for m in range(1, 18):  # based on each layer, 17 layers in total
        data = filtered_data[m-1]
        N = len(data)
        W = n * n  # Window size
        hd = 10 * 10000 // 520  # Sliding step size
        hdcs = (N - W) // hd + 1  # Number of sliding steps

        for k in range(1, hdcs + 1):
            one_data = data[(k-1) * hd: (k-1) * hd + W]
            # Normalize and scale the data to 0-255
            one_data = np.round((one_data - np.min(one_data)) / (np.max(one_data) - np.min(one_data)) * 255).astype(np.uint8)
            # Reshape the image
            one_data = one_data.reshape((n, n)) 
            
            data_all.append(one_data)
            # Save the image
            image_filename = f'p_{mcp_num}_{m}_{k}.png'
            image_path = os.path.join(folder, image_filename)
            Image.fromarray(one_data).save(image_path)
            
    print(filename, "Processing complete.")


### <center>[🏠 Home](../../../../../welcomePage.ipynb)</center>